# 🧪 Lab 1: Prompting an LLM with Python (Google Colab)

This lab shows how to call a Large Language Model (LLM) from Python, craft better prompts, and control generation using parameters like **temperature** and **max_tokens**.

### What you'll learn
1. How to set up environment and API keys in Colab
2. How to send your first prompt
3. How to improve prompts (role, constraints, few-shot examples)
4. How decoding parameters affect output
5. How to wrap calls in a reusable helper with retries

> **Why are we doing this?**
> In real engineering, LLMs act like an *API-first component*. Mastering prompts & parameters is the quickest way to get reliable, repeatable behavior for prototypes and production services.

## ✅ Step 0 — Colab Runtime Check

In [ ]:
import sys
print('Python', sys.version)
try:
    import google.colab  # type: ignore
    print('✅ Running in Google Colab')
except Exception:
    print('ℹ️ Not in Colab (that is okay for local runs).')

## 🔐 Step 1 — Install SDK & Set API Key

We will use the official **OpenAI Python SDK (>=1.0)**. You need an API key.

**In Colab:**
1. Go to [OpenAI API Keys](https://platform.openai.com/account/api-keys) and create a key.
2. Run the cell below — it will prompt you to paste the key.

We store the key in an environment variable to avoid hard-coding secrets.

In [ ]:
!pip -q install --upgrade openai>=1.40 matplotlib

import os
from getpass import getpass

if 'OPENAI_API_KEY' not in os.environ or not os.environ['OPENAI_API_KEY']:
    print('Enter your OpenAI API key (it will be hidden):')
    os.environ['OPENAI_API_KEY'] = getpass()

print('✅ API key loaded into environment (not printed).')

## 🧰 Step 2 — Minimal Client & Helper

In [ ]:
from openai import OpenAI
import time
from typing import List, Dict, Any

client = OpenAI()

DEFAULT_MODEL = 'gpt-4o-mini'  # fast & cost-efficient for labs

class LLMError(Exception):
    pass

def call_llm(messages: List[Dict[str, str]],
             model: str = DEFAULT_MODEL,
             temperature: float = 0.7,
             max_tokens: int = 400,
             retries: int = 2,
             **kwargs: Any) -> str:
    last_err = None
    for attempt in range(retries + 1):
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
                **kwargs
            )
            return resp.choices[0].message.content
        except Exception as e:
            last_err = e
            time.sleep(0.6 * (attempt + 1))
    raise LLMError(f'LLM call failed after retries: {last_err}')

## 🚀 Step 3 — Your First Prompt

In [ ]:
response = call_llm([
    {"role": "user", "content": "Explain Python decorators in simple terms with a short example."}
])
print(response)

## 🎭 Step 4 — Add a System Role

In [ ]:
response = call_llm([
    {"role": "system", "content": "You are a senior Python instructor. Be concise and precise."},
    {"role": "user", "content": "Explain Python generators with a tiny example."}
])
print(response)

## 📐 Step 5 — Add Constraints & Formatting

In [ ]:
prompt = (
    "Explain the Strategy design pattern for Python.\n"
    "Return exactly 3 bullet points and a 6-line code example."
)
response = call_llm([
    {"role": "system", "content": "You are a precise software architect."},
    {"role": "user", "content": prompt}
])
print(response)

## 🧩 Step 6 — Few-Shot Prompting (Examples)

In [ ]:
messages = [
    {"role": "system", "content": "You write compact, idiomatic Python."},
    {"role": "user", "content": "Task: Write a function with docstring that reverses a string."},
    {"role": "assistant", "content": """
"""Utilities for strings"""

def reverse_text(s: str) -> str:
    """Return the reverse of input string."""
    return s[::-1]
"""},
    {"role": "user", "content": "Task: Write a function with docstring that counts vowels in a string."}
]

print(call_llm(messages, temperature=0.2))

## 🎚️ Step 7 — Decoding Parameters: Temperature & Max Tokens

In [ ]:
question = "Give a one-sentence metaphor for microservices."
for t in [0.0, 0.4, 0.9]:
    txt = call_llm([
        {"role": "system", "content": "Be vivid but concise."},
        {"role": "user", "content": question}
    ], temperature=t, max_tokens=60)
    print(f"\nTemperature={t}:\n{txt}")

## 🧱 Step 8 — Robust Wrapper with JSON Output (Optional)

In [ ]:
import uuid, json

def call_llm_json(prompt: str, schema_hint: str) -> dict:
    trace_id = str(uuid.uuid4())
    sys_msg = (
        "You are a service that returns strictly valid JSON per the user's schema hint."
        f" Always return ONLY JSON. trace_id={trace_id}"
    )
    text = call_llm([
        {"role": "system", "content": sys_msg},
        {"role": "user", "content": f"Schema: {schema_hint}\nInput: {prompt}"}
    ], temperature=0.2, max_tokens=350)
    return json.loads(text)

print(call_llm_json(
    prompt="Generate 3 test cases for a login API with fields email and password.",
    schema_hint='{"tests":[{"name": "str", "input": {"email": "str", "password": "str"}}]}'
))

## 🧭 What to try next

1. Swap the task (e.g., summarize logs, propose test cases, write docstrings).
2. Change **temperature** and compare tone/variety.
3. Add **few-shot** examples for your team's preferred format.
4. Wrap `call_llm` with observability and rate-limit handling.

**You’ve completed Lab 1.** 🎉